In [8]:
import sys
import os
import glob
import re

import pandas as pd
import numpy as np
from tqdm import tqdm

In [10]:
df = pd.read_csv('../processed_data/train_v2.csv')
df.head(3)

,id,y,location,access,layout,age,direction,area,floor,bath_toilet,...,enviroment,structure,contact_period,max_floor,room_num,L,D,K,S,north
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,117,南東,20.01,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,...,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間,12,1,0,0,1,0,0
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,538,NaN,16.50,5,専用トイレ／\tシャワー／\t温水洗浄便座,...,【スーパー】 1283m,鉄骨造,2年間,10,1,0,0,0,0,0
2,3,110000,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,102,南,22.05,12,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,...,【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,RC（鉄筋コンクリート）,2年間,15,1,0,0,1,0,0


### structure の前処理

- コストの面からみた順序尺度が存在すると仮定

In [11]:
df['structure'].value_counts()

RC（鉄筋コンクリート）                   17505
木造                              5512
鉄骨造                             3726
SRC（鉄骨鉄筋コンクリート）                 2429
軽量鉄骨                            1812
ALC（軽量気泡コンクリート）                  297
その他                              136
PC（プレキャスト・コンクリート（鉄筋コンクリート））       46
HPC（プレキャスト・コンクリート（重量鉄骨））           4
ブロック                               3
Name: structure, dtype: int64

In [12]:
df['structure'].unique()

array(['RC（鉄筋コンクリート）', '鉄骨造', '木造', 'SRC（鉄骨鉄筋コンクリート）', '軽量鉄骨',
       'ALC（軽量気泡コンクリート）', 'その他', 'PC（プレキャスト・コンクリート（鉄筋コンクリート））',
       'HPC（プレキャスト・コンクリート（重量鉄骨））', 'ブロック'], dtype=object)

In [13]:
df['structure'].isnull().sum()

0

In [18]:
df.groupby(by='structure').describe()['y']

,count,mean,std,min,25%,50%,75%,max
structure,,,,,,,,
ALC（軽量気泡コンクリート）,297.0,107103.703704,33898.290135,48000.0,81000.0,102000.0,127000.0,255000.0
HPC（プレキャスト・コンクリート（重量鉄骨））,4.0,93000.000000,41044.691090,55000.0,69250.0,83500.0,107250.0,150000.0
PC（プレキャスト・コンクリート（鉄筋コンクリート））,46.0,106195.652174,21821.110640,69000.0,89250.0,102500.0,117250.0,159000.0
RC（鉄筋コンクリート）,17505.0,134366.134704,78147.007990,32000.0,92000.0,115000.0,155000.0,2500000.0
SRC（鉄骨鉄筋コンクリート）,2429.0,142525.408399,85043.488185,42000.0,91000.0,120000.0,165000.0,1120000.0
その他,136.0,81724.448529,22799.471945,36000.0,70000.0,78000.0,85625.0,200000.0
ブロック,3.0,60000.000000,15000.000000,45000.0,52500.0,60000.0,67500.0,75000.0
木造,5512.0,83823.202286,56896.651521,25000.0,57000.0,69000.0,90000.0,1450000.0
軽量鉄骨,1812.0,94458.160044,34477.836339,40000.0,70000.0,87000.0,113075.0,450000.0


- SRC, RC, ALC, PC, 軽量鉄骨,　HPC,　鉄骨造,その他・木造,ブロックの順に,8,7,6,5,4,3,2,1,0に置き換え

In [37]:
tmp = df['structure'].map({
    'SRC（鉄骨鉄筋コンクリート）': 8,
    'RC（鉄筋コンクリート）': 7,
    'ALC（軽量気泡コンクリート）': 6,
    'PC（プレキャスト・コンクリート（鉄筋コンクリート））': 5,
    '軽量鉄骨': 4,
    'HPC（プレキャスト・コンクリート（重量鉄骨））': 3,
    '鉄骨造':  2,
    'その他': 1,
    '木造': 1,
    'ブロック': 0
})

In [39]:
tmp.isnull().sum()

0

In [40]:
df['structure'] = tmp

In [42]:
df.to_csv('../processed_data/train_v3.csv')

In [43]:
test = pd.read_csv('../data/test.csv')
test.isnull().sum()

id           0
所在地          0
アクセス         0
間取り          0
築年数          0
方角        2808
面積           0
所在階          1
バス・トイレ     320
キッチン       635
放送・通信     3454
室内設備        73
駐車場       5356
周辺環境      9241
建物構造         0
契約期間      6981
dtype: int64

In [44]:
pd.read_csv('../hoge.csv').head()

,id,y,location,access,layout,age,direction,area,floor,bath_toilet,...,enviroment,structure,contact_period,max_floor,room_num,L,D,K,S,north
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,117,南東,20.01,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,...,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,7,2年間,12,1,0,0,1,0,0
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,538,NaN,16.50,5,専用トイレ／\tシャワー／\t温水洗浄便座,...,【スーパー】 1283m,2,2年間,10,1,0,0,0,0,0
2,3,110000,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,102,南,22.05,12,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,...,【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,7,2年間,15,1,0,0,1,0,0
3,4,150000,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,2LDK,352,南,60.48,3,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,...,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,7,2年間\t※この物件は\t定期借家\tです。,4,2,1,1,1,0,0
4,5,74000,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,2DK,379,南,39.66,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,...,【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,1,2年間,2,2,0,1,1,0,0
